In [4]:
import uproot
import awkward as ak
from particle import Particle

def pdg_to_name(pdgid: int) -> str:
    """Convert PDG ID to particle name if possible."""
    try:
        p = Particle.from_pdgid(pdgid)
        return p.name
    except Exception:
        return str(pdgid)  # fallback if not found

def build_children_map(mothers):
    """Build a dict: parent_idx -> list of child indices for a single event."""
    children = {i: [] for i in range(len(mothers))}
    for i, mom in enumerate(mothers):
        if mom >= 0:
            children[mom].append(i)
    return children

def get_decay_tree(pdgIds, status, pts, etas, phis, masses, children_map, idx, level=0):
    """Return tree-like decay structure with names, pdgId, status and 4-vector info."""
    pdgId = int(pdgIds[idx])
    st = int(status[idx])
    name = pdg_to_name(pdgId)
    indent = "  " * level

    # kinematics
    pt = float(pts[idx])
    eta = float(etas[idx])
    phi = float(phis[idx])
    mass = float(masses[idx])

    line = (
        f"{indent}{name} [{pdgId}] (status={st}) "
        f"pt={pt:.2f}, eta={eta:.2f}, phi={phi:.2f}, mass={mass:.2f}\n"
    )
    print("line: ",line)
    print("idx_0: ", idx)

    if idx in children_map:
        print("idx: ", idx)
        for c in children_map[idx]:
            print("children_map idx: ", children_map[idx])
            print("child: ", c)
            line += get_decay_tree(
                pdgIds, status, pts, etas, phis, masses, children_map, c, level + 1
            )
            print("line after child: ", line)

    return line

def dump_photon_chains(events, output_file="decay_chains_test.txt"):
    pdgIds = events["GenPart_pdgId"]
    status = events["GenPart_status"]
    mothers = events["GenPart_genPartIdxMother"]
    pts = events["GenPart_pt"]
    etas = events["GenPart_eta"]
    phis = events["GenPart_phi"]
    masses = events["GenPart_mass"]

    with open(output_file, "w") as f:
        for evt_idx in range(len(pdgIds)):
            pdg_evt = pdgIds[evt_idx]
            st_evt = status[evt_idx]
            mom_evt = mothers[evt_idx]
            pt_evt = pts[evt_idx]
            eta_evt = etas[evt_idx]
            phi_evt = phis[evt_idx]
            mass_evt = masses[evt_idx]

            children_map = build_children_map(mom_evt)
            print("Children map: ",children_map)

            # select photons in this event
            photons = ak.where(pdg_evt == 22)[0]
            for pho in photons:
                mom_idx = mom_evt[pho]
                if mom_idx >= 0 and pdg_evt[mom_idx] == 35 and st_evt[pho] == 23:
                    # climb up to top ancestor
                    root = mom_idx
                    print("root: ", root)
                    while root >= 0 and mom_evt[root] >= 0:
                        root = mom_evt[root]
                        print("climbing: ", root)

                    tree = get_decay_tree(
                        pdg_evt, st_evt, pt_evt, eta_evt, phi_evt, mass_evt,
                        children_map, root
                    )
                    print("final tree: ", tree)
                    f.write(f"\nEvent {evt_idx}\n")
                    f.write(tree)

if __name__ == "__main__":
    filename = "86D3D6A3-EE02-5849-8FC0-C89B9BE7D930.root"
    events = uproot.open(filename)["Events"].arrays(
        [
            "GenPart_pdgId", "GenPart_status", "GenPart_genPartIdxMother",
            "GenPart_pt", "GenPart_eta", "GenPart_phi", "GenPart_mass"
        ],
        library="ak"
    )

    dump_photon_chains(events[:10])

Children map:  {0: [2, 3], 1: [], 2: [4], 3: [5], 4: [6], 5: [7], 6: [8, 9], 7: [10, 11], 8: [], 9: [], 10: [12, 13], 11: [14, 15], 12: [16], 13: [17, 18, 19, 24, 25], 14: [], 15: [], 16: [], 17: [], 18: [20], 19: [21], 20: [22, 23], 21: [26], 22: [], 23: [], 24: [], 25: [], 26: []}
Children map:  {0: [2, 3], 1: [], 2: [4], 3: [5], 4: [6], 5: [7], 6: [8], 7: [9], 8: [10], 9: [11], 10: [12], 11: [13], 12: [14, 15], 13: [19, 20], 14: [16, 17], 15: [18], 16: [], 17: [], 18: [], 19: [21, 22], 20: [23, 24], 21: [25], 22: [26, 27, 28], 23: [], 24: [], 25: [], 26: [], 27: [29], 28: [30], 29: [31], 30: [], 31: []}
Children map:  {0: [2, 3], 1: [], 2: [4], 3: [5], 4: [6], 5: [7], 6: [8], 7: [9], 8: [10], 9: [11], 10: [12, 13], 11: [17, 18], 12: [14, 15], 13: [16], 14: [], 15: [], 16: [], 17: [19, 20], 18: [21, 22], 19: [23, 26, 28, 29], 20: [], 21: [], 22: [], 23: [27], 24: [42], 25: [], 26: [], 27: [], 28: [43], 29: [44], 30: [45, 46], 31: [], 32: [], 33: [], 34: [47, 48], 35: [], 36: [], 37: 